In [87]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

class Layer():
    def __init__(self, input_dim, output_dim):
        self.layer = nn.Linear(input_dim, output_dim)
        self.activations = None
        #self.mean = np.zeros((input_dim, output_dim))
        #self.variance = np.zeros((input_dim, output_dim))
        
    def forward(self, x):
        self.activations = self.layer(x)
        #self.variance = np.var(activations.detach().numpy(), axis = 0)
        #self.mean = np.mean(activations.detach().numpy(), axis = 0)
        return self.activations
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = Layer(320, 150)
        self.fc2 = Layer(150, 150)
        self.fc3 = Layer(150, 150)
        self.fc4 = Layer(150, 150)
        self.fc5 = Layer(150, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1.forward(x))
        x = F.relu(self.fc2.forward(x))
        x = F.relu(self.fc3.forward(x))
        x = F.relu(self.fc4.forward(x))
        x = self.fc5.forward(x)
        return F.log_softmax(x, dim=1)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [88]:
torch.manual_seed(1)

device = torch.device("cpu")

kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)



In [89]:
for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

val_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10000, shuffle=True, **kwargs)
test(model, device, val_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.293017
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.296916
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.320870
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.305141
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.306541
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.302220
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.296724
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.297585
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.306103
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.304100
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.297133
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.303020
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.312071
Train Epoch: 0 [8320/60000 (14%)]	Loss: 2.300865
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.301143
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.314250
Train Epoch: 0 [10240/60000 (17%)]	Loss: 2.302002
Train Epoch: 0 [10880/60000 (18%)]	Loss: 2.300359
Train Epoch: 0 [11520/60000 (19%)]	Loss: 2.301488
Train Epoch: 0 [12160/60000 (20%)]	Loss: 2.307114
Train Epoch: 0 [12800/60000 (

Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.281868
Train Epoch: 1 [45440/60000 (76%)]	Loss: 2.306850
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.307101
Train Epoch: 1 [46720/60000 (78%)]	Loss: 2.303783
Train Epoch: 1 [47360/60000 (79%)]	Loss: 2.315664
Train Epoch: 1 [48000/60000 (80%)]	Loss: 2.301458
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.299720
Train Epoch: 1 [49280/60000 (82%)]	Loss: 2.305551
Train Epoch: 1 [49920/60000 (83%)]	Loss: 2.311285
Train Epoch: 1 [50560/60000 (84%)]	Loss: 2.301457
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.298064
Train Epoch: 1 [51840/60000 (86%)]	Loss: 2.298581
Train Epoch: 1 [52480/60000 (87%)]	Loss: 2.314017
Train Epoch: 1 [53120/60000 (88%)]	Loss: 2.289116
Train Epoch: 1 [53760/60000 (90%)]	Loss: 2.288137
Train Epoch: 1 [54400/60000 (91%)]	Loss: 2.298108
Train Epoch: 1 [55040/60000 (92%)]	Loss: 2.292159
Train Epoch: 1 [55680/60000 (93%)]	Loss: 2.305802
Train Epoch: 1 [56320/60000 (94%)]	Loss: 2.297432
Train Epoch: 1 [56960/60000 (95%)]	Loss: 2.308892


Train Epoch: 3 [28800/60000 (48%)]	Loss: 2.311588
Train Epoch: 3 [29440/60000 (49%)]	Loss: 2.282167
Train Epoch: 3 [30080/60000 (50%)]	Loss: 2.282946
Train Epoch: 3 [30720/60000 (51%)]	Loss: 2.281618
Train Epoch: 3 [31360/60000 (52%)]	Loss: 2.298720
Train Epoch: 3 [32000/60000 (53%)]	Loss: 2.287293
Train Epoch: 3 [32640/60000 (54%)]	Loss: 2.292427
Train Epoch: 3 [33280/60000 (55%)]	Loss: 2.289156
Train Epoch: 3 [33920/60000 (57%)]	Loss: 2.277247
Train Epoch: 3 [34560/60000 (58%)]	Loss: 2.297415
Train Epoch: 3 [35200/60000 (59%)]	Loss: 2.281999
Train Epoch: 3 [35840/60000 (60%)]	Loss: 2.288051
Train Epoch: 3 [36480/60000 (61%)]	Loss: 2.296229
Train Epoch: 3 [37120/60000 (62%)]	Loss: 2.297471
Train Epoch: 3 [37760/60000 (63%)]	Loss: 2.268244
Train Epoch: 3 [38400/60000 (64%)]	Loss: 2.284973
Train Epoch: 3 [39040/60000 (65%)]	Loss: 2.272146
Train Epoch: 3 [39680/60000 (66%)]	Loss: 2.278913
Train Epoch: 3 [40320/60000 (67%)]	Loss: 2.299806
Train Epoch: 3 [40960/60000 (68%)]	Loss: 2.281532


Train Epoch: 5 [12800/60000 (21%)]	Loss: 2.259926
Train Epoch: 5 [13440/60000 (22%)]	Loss: 2.310827
Train Epoch: 5 [14080/60000 (23%)]	Loss: 2.245170
Train Epoch: 5 [14720/60000 (25%)]	Loss: 2.299317
Train Epoch: 5 [15360/60000 (26%)]	Loss: 2.260689
Train Epoch: 5 [16000/60000 (27%)]	Loss: 2.323350
Train Epoch: 5 [16640/60000 (28%)]	Loss: 2.262460
Train Epoch: 5 [17280/60000 (29%)]	Loss: 2.262241
Train Epoch: 5 [17920/60000 (30%)]	Loss: 2.250052
Train Epoch: 5 [18560/60000 (31%)]	Loss: 2.265228
Train Epoch: 5 [19200/60000 (32%)]	Loss: 2.240309
Train Epoch: 5 [19840/60000 (33%)]	Loss: 2.273073
Train Epoch: 5 [20480/60000 (34%)]	Loss: 2.212702
Train Epoch: 5 [21120/60000 (35%)]	Loss: 2.276664
Train Epoch: 5 [21760/60000 (36%)]	Loss: 2.267365
Train Epoch: 5 [22400/60000 (37%)]	Loss: 2.278475
Train Epoch: 5 [23040/60000 (38%)]	Loss: 2.256415
Train Epoch: 5 [23680/60000 (39%)]	Loss: 2.265098
Train Epoch: 5 [24320/60000 (41%)]	Loss: 2.276303
Train Epoch: 5 [24960/60000 (42%)]	Loss: 2.255002


Train Epoch: 6 [57600/60000 (96%)]	Loss: 2.230738
Train Epoch: 6 [58240/60000 (97%)]	Loss: 2.230556
Train Epoch: 6 [58880/60000 (98%)]	Loss: 2.233315
Train Epoch: 6 [59520/60000 (99%)]	Loss: 2.218423

Test set: Average loss: 2.2138, Accuracy: 2580/10000 (26%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 2.196932
Train Epoch: 7 [640/60000 (1%)]	Loss: 2.261824
Train Epoch: 7 [1280/60000 (2%)]	Loss: 2.243228
Train Epoch: 7 [1920/60000 (3%)]	Loss: 2.189883
Train Epoch: 7 [2560/60000 (4%)]	Loss: 2.228045
Train Epoch: 7 [3200/60000 (5%)]	Loss: 2.226187
Train Epoch: 7 [3840/60000 (6%)]	Loss: 2.206724
Train Epoch: 7 [4480/60000 (7%)]	Loss: 2.234978
Train Epoch: 7 [5120/60000 (9%)]	Loss: 2.292228
Train Epoch: 7 [5760/60000 (10%)]	Loss: 2.264842
Train Epoch: 7 [6400/60000 (11%)]	Loss: 2.251003
Train Epoch: 7 [7040/60000 (12%)]	Loss: 2.197531
Train Epoch: 7 [7680/60000 (13%)]	Loss: 2.297596
Train Epoch: 7 [8320/60000 (14%)]	Loss: 2.252627
Train Epoch: 7 [8960/60000 (15%)]	Loss: 2.234956
Train Epoch: 7 [9

Train Epoch: 8 [41600/60000 (69%)]	Loss: 2.188683
Train Epoch: 8 [42240/60000 (70%)]	Loss: 2.219256
Train Epoch: 8 [42880/60000 (71%)]	Loss: 2.265642
Train Epoch: 8 [43520/60000 (72%)]	Loss: 2.215549
Train Epoch: 8 [44160/60000 (74%)]	Loss: 2.196834
Train Epoch: 8 [44800/60000 (75%)]	Loss: 2.201407
Train Epoch: 8 [45440/60000 (76%)]	Loss: 2.192259
Train Epoch: 8 [46080/60000 (77%)]	Loss: 2.260823
Train Epoch: 8 [46720/60000 (78%)]	Loss: 2.166126
Train Epoch: 8 [47360/60000 (79%)]	Loss: 2.201839
Train Epoch: 8 [48000/60000 (80%)]	Loss: 2.156651
Train Epoch: 8 [48640/60000 (81%)]	Loss: 2.244804
Train Epoch: 8 [49280/60000 (82%)]	Loss: 2.169443
Train Epoch: 8 [49920/60000 (83%)]	Loss: 2.211060
Train Epoch: 8 [50560/60000 (84%)]	Loss: 2.210292
Train Epoch: 8 [51200/60000 (85%)]	Loss: 2.212338
Train Epoch: 8 [51840/60000 (86%)]	Loss: 2.245151
Train Epoch: 8 [52480/60000 (87%)]	Loss: 2.164019
Train Epoch: 8 [53120/60000 (88%)]	Loss: 2.229444
Train Epoch: 8 [53760/60000 (90%)]	Loss: 2.134171


In [90]:
activ1 = model.fc1.activations.detach().numpy()
activ2 = model.fc2.activations.detach().numpy()
activ3 = model.fc3.activations.detach().numpy()
activ4 = model.fc4.activations.detach().numpy()
activ5 = model.fc5.activations.detach().numpy()

In [96]:
activs = [activ1,activ2,activ3,activ4,activ5]
for activ in activs:
    corr = np.corrcoef(activ.T)
    mask = np.ones(corr.shape, dtype=bool)
    np.fill_diagonal(mask, 0)
    min_value = np.min(corr)
    max_value = np.max(np.abs(corr[mask]))
    print(min_value,max_value)
    print("var",np.var(corr))

(-0.8620582286784375, 0.8743810710057301)
('var', 0.09899516857016831)
(-0.8889741863830612, 0.8889741863830612)
('var', 0.11225490545324895)
(-0.8857556209722355, 0.8857556209722355)
('var', 0.13103510035661917)
(-0.9072437035732301, 0.9072437035732301)
('var', 0.1499160475278939)
(-0.842941735576707, 0.842941735576707)
('var', 0.24103884759927077)


In [43]:
print(model.fc3.mean)
print(model.fc3.variance)

[-0.89464617 -0.15404972  0.2182424  -0.04803471  0.612027    0.78216285
  0.23039587 -0.27611163  0.46616864 -0.47589928]
[4.409854  3.1290188 2.6786973 2.924032  2.250953  0.8600879 2.381398
 4.1028824 1.2036417 2.7671638]
